In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Clustering using K-Means_Xinghe Huang').getOrCreate()

In [3]:
df = spark.read.load("./Customer list.csv",format="csv",header="true")

In [4]:
df.show()

+----------+------------+------+-----------------------------------+----------+----+---+--------------------+---------------------+-----------------+------------------+--------+------+--------------------+--------+-----+---------+------------------+----+--------+
|first_name|   last_name|gender|past_3_years_bike_related_purchases|       DOB| _c5|Age|           job_title|job_industry_category|   wealth_segment|deceased_indicator|owns_car|tenure|             address|postcode|state|  country|property_valuation|Rank|   Value|
+----------+------------+------+-----------------------------------+----------+----+---+--------------------+---------------------+-----------------+------------------+--------+------+--------------------+--------+-----+---------+------------------+----+--------+
|   Chickie|     Brister|  Male|                                 86|12-07-1957|1957| 64|     General Manager|        Manufacturing|    Mass Customer|                 N|     Yes|    14|    45 Shopko Center|   

In [10]:
df.describe().show()

+-------+----------+--------------------+------+-----------------------------------+----------+------------------+------------------+-------------------+---------------------+-----------------+------------------+--------+----------------+-------------+-----------------+-----+---------+------------------+------------------+------------------+
|summary|first_name|           last_name|gender|past_3_years_bike_related_purchases|       DOB|               _c5|               Age|          job_title|job_industry_category|   wealth_segment|deceased_indicator|owns_car|          tenure|      address|         postcode|state|  country|property_valuation|              Rank|             Value|
+-------+----------+--------------------+------+-----------------------------------+----------+------------------+------------------+-------------------+---------------------+-----------------+------------------+--------+----------------+-------------+-----------------+-----+---------+------------------+-------

In [11]:
df.printSchema()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- past_3_years_bike_related_purchases: string (nullable = true)
 |-- DOB: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- job_industry_category: string (nullable = true)
 |-- wealth_segment: string (nullable = true)
 |-- deceased_indicator: string (nullable = true)
 |-- owns_car: string (nullable = true)
 |-- tenure: string (nullable = true)
 |-- address: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- property_valuation: string (nullable = true)
 |-- Rank: string (nullable = true)
 |-- Value: string (nullable = true)



In [12]:
from pyspark.sql.types import (StructField,StringType,IntegerType,StructType)

In [13]:
data_schema = [StructField('age',IntegerType(),True),
              StructField('name',StringType(),True)]

final_struct = StructType(fields=data_schema)

In [14]:
df.na.drop().show()

+----------+------------+------+-----------------------------------+----------+----+---+--------------------+---------------------+-----------------+------------------+--------+------+--------------------+--------+-----+---------+------------------+----+--------+
|first_name|   last_name|gender|past_3_years_bike_related_purchases|       DOB| _c5|Age|           job_title|job_industry_category|   wealth_segment|deceased_indicator|owns_car|tenure|             address|postcode|state|  country|property_valuation|Rank|   Value|
+----------+------------+------+-----------------------------------+----------+----+---+--------------------+---------------------+-----------------+------------------+--------+------+--------------------+--------+-----+---------+------------------+----+--------+
|   Chickie|     Brister|  Male|                                 86|12-07-1957|1957| 64|     General Manager|        Manufacturing|    Mass Customer|                 N|     Yes|    14|    45 Shopko Center|   

In [15]:
df.count()

1000

In [17]:
df.na.drop().count()

852

In [18]:
df.printSchema()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- past_3_years_bike_related_purchases: string (nullable = true)
 |-- DOB: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- job_industry_category: string (nullable = true)
 |-- wealth_segment: string (nullable = true)
 |-- deceased_indicator: string (nullable = true)
 |-- owns_car: string (nullable = true)
 |-- tenure: string (nullable = true)
 |-- address: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- property_valuation: string (nullable = true)
 |-- Rank: string (nullable = true)
 |-- Value: string (nullable = true)



In [19]:
df.select('age').show()

+---+
|age|
+---+
| 64|
| 63|
| 62|
| 61|
| 60|
| 59|
| 58|
| 57|
| 56|
| 55|
| 54|
| 53|
| 52|
| 51|
| 50|
| 49|
| 48|
| 47|
| 83|
| 67|
+---+
only showing top 20 rows



In [20]:
ageColumn = df.select('age')

ageColumn.show()

+---+
|age|
+---+
| 64|
| 63|
| 62|
| 61|
| 60|
| 59|
| 58|
| 57|
| 56|
| 55|
| 54|
| 53|
| 52|
| 51|
| 50|
| 49|
| 48|
| 47|
| 83|
| 67|
+---+
only showing top 20 rows



In [21]:
df.withColumn('double_age',df['age']*2).show()

+----------+------------+------+-----------------------------------+----------+----+---+--------------------+---------------------+-----------------+------------------+--------+------+--------------------+--------+-----+---------+------------------+----+--------+----------+
|first_name|   last_name|gender|past_3_years_bike_related_purchases|       DOB| _c5|Age|           job_title|job_industry_category|   wealth_segment|deceased_indicator|owns_car|tenure|             address|postcode|state|  country|property_valuation|Rank|   Value|double_age|
+----------+------------+------+-----------------------------------+----------+----+---+--------------------+---------------------+-----------------+------------------+--------+------+--------------------+--------+-----+---------+------------------+----+--------+----------+
|   Chickie|     Brister|  Male|                                 86|12-07-1957|1957| 64|     General Manager|        Manufacturing|    Mass Customer|                 N|     Ye

In [22]:
df.createOrReplaceTempView('people')

In [23]:
results = spark.sql("SELECT * FROM people")

In [24]:
results = spark.sql("SELECT age FROM people WHERE age >= 19")
results.show()

+---+
|age|
+---+
| 64|
| 63|
| 62|
| 61|
| 60|
| 59|
| 58|
| 57|
| 56|
| 55|
| 54|
| 53|
| 52|
| 51|
| 50|
| 49|
| 48|
| 47|
| 83|
| 67|
+---+
only showing top 20 rows



In [25]:
df.na.drop(how="all").show()

+----------+------------+------+-----------------------------------+----------+----+---+--------------------+---------------------+-----------------+------------------+--------+------+--------------------+--------+-----+---------+------------------+----+--------+
|first_name|   last_name|gender|past_3_years_bike_related_purchases|       DOB| _c5|Age|           job_title|job_industry_category|   wealth_segment|deceased_indicator|owns_car|tenure|             address|postcode|state|  country|property_valuation|Rank|   Value|
+----------+------------+------+-----------------------------------+----------+----+---+--------------------+---------------------+-----------------+------------------+--------+------+--------------------+--------+-----+---------+------------------+----+--------+
|   Chickie|     Brister|  Male|                                 86|12-07-1957|1957| 64|     General Manager|        Manufacturing|    Mass Customer|                 N|     Yes|    14|    45 Shopko Center|   

In [32]:
import pandas

demographic=pandas.read_csv("customer demographic.csv")
print(demographic.head())
address=pandas.read_csv("customer address.csv")
print(address.head())

merger = pandas.merge(demographic, address, on=['customer_id'])
print(merger.head())

   customer_id      first_name  last_name  gender  \
0            1         Laraine  Medendorp       F   
1            2             Eli    Bockman    Male   
2            3           Arlin     Dearle    Male   
3            4          Talbot        NaN    Male   
4            5  Sheila-kathryn     Calton  Female   

   past_3_years_bike_related_purchases         DOB               job_title  \
0                                   93  1953/10/12     Executive Secretary   
1                                   81  1980/12/16  Administrative Officer   
2                                   61   1954/1/20      Recruiting Manager   
3                                   33   1961/10/3                     NaN   
4                                   56   1977/5/13           Senior Editor   

  job_industry_category     wealth_segment deceased_indicator  \
0                Health      Mass Customer                  N   
1    Financial Services      Mass Customer                  N   
2              Pr